# Construcción de un detector de spam
Construcción de un detector de spam en textos cortos (SMS) en inglés:
Se han realizado estudios y se ha determinado que los mensajes de spam contienen palabras como free, win, winner, cash y prize; además suelen incluir letras mayúsculas y signos de admiración.
Este es un problema de clasificación supervisada binaria, dado que los mensajes serán identificados como “spam”o“no spam” y los datos de entrenamiento están etiquetados, se encuentran en
https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.
Como el conjunto de datos es grande (5572 filas) y el modelo sólo acepta valores numéricos
como entrada, es necesario hacer un preprocesamiento; utilizaremos el concepto de Bag of Words
(BoW bolsa de palabras). BoW es un término usado para especificar problemas que tienen datos
de texto en los que se requiere obtener la frecuencia de cada palabra en el texto tratando cada
palabra como un elemento independiente sin importar el orden.
Para esto puede usarse la clase CountVectorizer de sklearn, esta clase realiza lo siguiente:

- Crea tokens individuales para cada palabra asignando un ID entero para cada token
- Cuenta las ocurrencias de cada token
- Convierte todas las palabras en minúsculas para no tratar la misma palabra como dos
distintas si están escritas con alguna letra mayúscula
- Ignora los signos de puntuación para no tratar como distintas aquellas palabras que incluyan
alguno de estos símbolos
- Un parámetro importante es stop_words que sirve para no tomar en cuenta las palabras
más comúnes del idioma especificado; p.e. si se indica english se ignoraran entradas como
“am”, “and”, “the”, etc.
Como ejemplo sencillo, se consideran cuatro documentos y se aplica CountVectorizer :

In [22]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']                                       

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
print(count)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [24]:
count.fit(documents)
names = count.get_feature_names()
names

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

Son todas las palabras únicas del texto

In [25]:
count.transform(documents).toarray()

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

Indica las ocurrencias de cada palabra

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
df = pd.read_csv('https://bit.ly/2kCy3CN',
                 sep='\t',
                 names=['label', 'sms_message'])
df.head(3)

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


Etiquetas
ham - mensaje normal
spam - span

In [28]:
# Preprocesamiento
# Cambiar a etiquetas de tipo númerico

df.label = df.label.map({'ham':0, 'spam':1})
df.head(3)

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


**Cuando tenemos conjuntos etiquetado, podemos dividir en conjunto de entramientos y prueba**

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.sms_message, df.label,
                                                    random_state=1)
print('Completo: ', df.shape[0])
print('Entrenamiento: ', X_train.shape[0])
print('Prueba: ', X_test.shape[0])

Completo:  5572
Entrenamiento:  4179
Prueba:  1393


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorize = CountVectorizer()
X_train_count = count_vectorize.fit_transform(X_train) 
X_test_count = count_vectorize.transform(X_test) 

fit- es ajusta, es aplicar el algoritmo, en ese caso, sacar la cuenta de ocucurrencia de cada palabra

fit_transform - obtiene las cuentas (frecuencias) y devuelve el el valor (aplica al mismo objeto)

transfom- una vez entrenado tu modelo, se usa transform para aplicarlo 

In [31]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
# Obtenemos las probabilidades
nb.fit(X_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
y_hat = nb.predict(X_test_count)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy: ', accuracy_score(y_test, y_hat))
print('Precision: ', precision_score(y_test, y_hat))
print('Recall: ', recall_score(y_test, y_hat))
print('F1: ', f1_score(y_test, y_hat))

Accuracy:  0.9885139985642498
Precision:  0.9720670391061452
Recall:  0.9405405405405406
F1:  0.9560439560439562
